In [63]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

import pandas as pd

pd.set_option('display.max_colwidth', None)

In [64]:
df_orig = pd.read_csv('files/items_titles.csv')
print(len(df_orig))
df_orig.head(10)

30000


,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Vermelho
1,Tenis Para Caminhada Super Levinho Spider Corrida
2,Tênis Feminino Le Parc Hocks Black/ice Original Envio Já
3,Tênis Olympikus Esportivo Academia Nova Tendência Triunfo
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicleta Carregáve
5,Tênis Casual Masculino Zarato 941 Preto 632
6,Tênis Infantil Ortopasso Conforto Jogging
7,Tv Samsung Qled 8k Q800t Semi Nova
8,Tênis Usthemp Short Temático - Maria Vira-lata 2
9,Sapatênis West Coast Urban Couro Masculino


In [65]:
df_test = pd.read_csv('items_titles_test.csv')
print(len(df_test))
df_test.head(10)

10000


,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C/ Rolamento
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde
5,Tênis Oakley Frequency 3.0 Preto/marrom
6,Tênis Jogging Feminino Premium Super Lançamento Vizzano
7,Under Armour Hovr Phantom 2 Conexão Bluetooth Tênis Running
8,Tenis Infantil Feminino Menina Criança Moça
9,Tênis Labellamafia Saturn 6 Cores Disponíveis


# SKLEARN TEXT SIMMILARITY

In [66]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [67]:
def sklearn_text_similarity(text1, text2):
    # Convert the texts into TF-IDF vectors
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([text1, text2])

    # Calculate the cosine similarity between the vectors
    similarity = cosine_similarity(vectors)
    return similarity[0][1]

In [68]:
text1 = 'Zapatillas Nike'
text2 = 'Zapatillas Adidas'

sklearn_text_simmilarity(text1, text2)

0.33609692727625756

# Manual Process

In [69]:
from unidecode import unidecode
import re

In [70]:
# Clean text from non alpha characters
def clean_text(text):
    new_text = unidecode(text)                              # Replace special characters
    new_text = new_text.lower()                             # Lowercase characters
    new_text = new_text.replace('/',' ')                    # Replace '/' with a space (sometimes it separates two words without any spacing)
    new_text = re.sub(r'[^A-Za-z0-9 ]+', '', new_text)      # Removing all non alphanumeric characters
    new_text = re.sub(r'[0-9]+', '', new_text)              # Removing all numbers
    return new_text

# Split text into list of words, removing elements with less than 3 characters
def split_text(text):
    splitted_text = text.split(' ')
    splitted_text_clean = list(filter(lambda x: len(x)>2, splitted_text))
    return splitted_text_clean


# Put it all together
def product_similarity_score(text1, text2):
    # Let's start by cleaning and splitting text1
    text_1 = clean_text(text1)
    text_1_list = split_text(text_1)

    # Now let's clean and split text2
    text_2 = clean_text(text2)
    text_2_list = split_text(text_2)

    # We now proceed to create an intersection set and its length
    text_clean_intersection_set = set(text_1_list).intersection(text_2_list)
    text_clean_intersection_set_len = len(text_clean_intersection_set)

    # To calculate the score we need also the total number of words (with duplicates) in both text lists
    text_clean_1_2_all_words_len = len(text_1_list)+len(text_2_list)

    # Here we want to know what percentage of all words in both lists
    try:
        score = (text_clean_intersection_set_len*2)/text_clean_1_2_all_words_len
    except ZeroDivisionError:
        score = 0

    return score

# Let's test this out with the example that was given in the task description

text1 = 'zapatílLas  Nike'
text2 = 'Zapatillas Adidas'

scr = product_similarity_score(text1, text2)
print(scr)


0.5


Now we have to apply this to a whole dataframe with 10k observations and compare them all with themselves

In [71]:
df_test.head()

,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C/ Rolamento
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde


In [72]:
text1_list = []
text2_list = []
score_list = []

for text1 in df_test.iloc[:,0]:
    for text2 in df_test.iloc[:,0]:
        scr = product_similarity_score(text1, text2)
        if scr > 0:
            text1_list.append(text1)
            text2_list.append(text2)
            score_list.append(scr)

print('Done!')

Done!


It took the loop 51 minutes and 25.7 secondes to iterate through every combination of the test dataframen and itself 
- 10,000**2 = 100,000,000 combinations

Let's create a dataframe with the results...

In [73]:
product_similarity_df = pd.DataFrame(
    {'ITE_ITEM_TITLE_1': text1_list,
     'ITE_ITEM_TITLE_2': text2_list,
     'Score': score_list
     })

Let's see some of the similarities that were caught in the loop:

In [82]:
# RANDOM SAMPLE OF 30
product_similarity_df.sample(n=30).round(2).head(30)

,ITE_ITEM_TITLE_1,ITE_ITEM_TITLE_2,Score
803308,Skechers Tenis Feminino 12985,Tênis Feminino Moleca Plataforma Preto,0.50
30850250,Pe Com Pe Tênis Runner Feminino Pink Com Marinho + Brinde,Tênis Usthemp Short Temático - Unicórnio 3 Cz,0.15
51578006,Tênis Quiz Plataforma Feminino Flatform Sem Cadarço Conforto,Tênis Feminino Sapatenis Casual Estiloso Confort | P01.osn22,0.27
22855107,Sapatos Respiráveis Masculinos Plus Size Casual All-match [u,Sapatos Crianças Confortáveis Tênis Sapatos Casuais,0.15
40815266,Tênis Preto Masculino Olympikus Oasis Original Frete Grátis!,Tênis Destroyer Fem Country Bordado De Ferradura Lançamento,0.13
52870796,Tênis Molekinho Tecido Mexico/multi Cinza/cinza,Tênis Qix 80s,0.22
50146959,Tênis Hocks Pro Model Solo Black Camuflado Marcelo Formiga,Tênis Skechers Go Walk Max Clinched - Masculino,0.13
36059081,Tênis Feminino Olympikus Gravidade Marinho,Sapatênis Feminino Bege Tênis Marrom Slip On Confort Yone,0.31
33472223,Tenis Flatform Mississipi Q4691 Feminina,Tênis Masculino Esportivo Twist Olympikus,0.22
29245008,Tenis Marinho De Marca Tenis De Marca Com Led Black Friday,Tênis Feminino Rosa/azul Plataforma Tie Dye Via Marte,0.11


In [83]:
# Highest Score (30 cases) - Does not include perfect 1 scores (probably same item)
product_similarity_df[product_similarity_df['Score']<1].sort_values('Score', ascending=False).round(2).head(30)

,ITE_ITEM_TITLE_1,ITE_ITEM_TITLE_2,Score
26387017,Mountain Bike Sense Fun Comp 2021/22 2021 Aro 29 M 16v Freios De Disco Hidráulico Câmbios Shimano Altus M315-ts Y Shimano Tourney Tx800 Cor Cinza/roxo,Mountain Bike Sense Fun Comp 2021/22 2021 Aro 29 M 16v Freios De Disco Hidráulico Câmbios Shimano Altus M315-ts Y Shimano Tourney Tx800 Cor Cinza/roxo,0.94
54801641,Tenis All Star Chuck Taylor Lilas Prata - Serie Limitada,Tenis All Star Chuck Taylor Prata - Serie Limitada,0.94
11585031,Tenis All Star Chuck Taylor Prata - Serie Limitada,Tenis All Star Chuck Taylor Lilas Prata - Serie Limitada,0.94
5462253,Tênis Fila Racer For All Feminino Corrida - Caminhada,Tenis Feminino Corrida Caminhada Running Fila Racer For All,0.94
11601513,Mountain Bike Sutton New Aro 29 19 27v Freios De Disco Hidráulico Câmbios Shimano Tourney Y Shimano Altus Cor Preto/laranja/prateado,Mountain Bike Sutton New Aro 29 19 27v Freios De Disco Hidráulico Câmbios Shimano Tourney Y Shimano Altus Cor Preto/laranja/prateado,0.94
20657667,Tenis Feminino Corrida Caminhada Running Fila Racer For All,Tênis Fila Racer For All Feminino Corrida - Caminhada,0.94
48218128,Tênis Adulto Polo Farm Casual Masculino Calce Facil,Tênis Adulto Slip On Polo Farm Casual Masculino Calce Facil,0.94
25893597,Tênis Adulto Slip On Polo Farm Casual Masculino Calce Facil,Tênis Adulto Polo Farm Casual Masculino Calce Facil,0.94
54008503,Mountain Bike Dropp Bikes Z7-x Aro 29 19 27v Freios De Disco Hidráulico Câmbios Dropp Cor Cinza/vermelho Com Descanso Lateral,Mountain Bike Dropp Bikes Z7-x Aro 29 19 27v Freios De Disco Hidráulico Câmbios Dropp Cor Cinza/vermelho Com Descanso Lateral,0.94
22816134,Mountain Bike Caloi Vulcan Aro 29 17 21v Freios De Disco Mecânico Câmbios Sunrun Dual Pull Y Sunrun 26a Cor Branco/vermelho,Mountain Bike Caloi Vulcan Aro 29 17 21v Freios De Disco Mecânico Câmbios Sunrun Dual Pull Y Sunrun 26a Cor Branco/vermelho,0.94


In [84]:
# Around 50% similarity (30 cases)
product_similarity_df[product_similarity_df['Score']<=0.5].sort_values('Score', ascending=False).round(2).head(30)

,ITE_ITEM_TITLE_1,ITE_ITEM_TITLE_2,Score
42082877,Tênis Feminino Casual De Cadarço Super Confortável Luxo 302,Tênis Feminino Casual Branco Plataforma,0.5
31325764,Kit 2 Pares Tenis Botinha Masculino Polo Casual Original,Kit 3 Sapatênis Casual Masculino,0.5
3702914,Tênis Klin Infantil Sport Menino Masculino Volta Ás Aulas,Tenis Infantil Masculino Skate Brinde Tênis Menino Jogging,0.5
14429879,Tênis Bull Terrrier Tecno Ii Preto Masculino,Tênis Hurley Hur0005 Masculino Preto/ciano,0.5
9805047,Tênis Feminino Viena Modare,Tenis Feminino Ramarim Branco - 20802,0.5
23873988,Tênis Feminino P/ Faculdade Moleca Lona Sider Lançamento!!!,Tênis Feminino Preto Moleca Original 5296155,0.5
36044727,Tênis Cole Haan Preto Tam 35,Tênis Dakota Preto G0841,0.5
9805031,Tênis Feminino Viena Modare,Tênis Mule Feminino Casual,0.5
30146269,Tênis Olympikus Triunfo Feminino,Tenis Olympikus Easy Ref.: 43259650,0.5
20551305,Tênis Juliette Via Marte Original,Tênis Jordan Original,0.5


To end the notebook, let's see some cases that are higher than a similarity threshold, say, 0.5.

In [86]:
# RANDOM SAMPLE OF 30
product_similarity_df[(product_similarity_df['Score']>=0.5)&(product_similarity_df['Score']<1)].sample(30).round(2).head(30)

,ITE_ITEM_TITLE_1,ITE_ITEM_TITLE_2,Score
33305278,Kit 2 Tênis Caminhada Academia Training Leve Feminino,Tênis Feminino Academia Caminhada Confortável E Macio 02,0.62
39312158,Tênis Usthemp One Temático - Cavalo Árabe,Tênis Usthemp Slip-on Temático - Dark Unicorn 2,0.50
55086729,Tenis Actvitta Masculino Azul,Tênis Moderno Masculino Original,0.50
44581588,Tênis Feminino Ramarim Flatform Napa 2180104,Slip On Tênis Moleca Flatform Sem Cadarço 5658. Feminino,0.50
1820516,Tênis Esportivo Olympikus Masculino Sonoro 799 Branco,Tênis Masculino Olympikus Evidence Marinho,0.55
8103214,Tenis Para Meninas Com Luzinha Pisca Pisca E Bolsinha Lol,Tenis Com Luzinha Que Pisca Do Batman,0.53
39662200,Sapatênis Pegada Casual Couro 119001-06,Sapatênis Casual Masculino Couro Legítimo Costurado Oferta,0.55
19512259,Tenis Feminino Casual Skynner S1100,Tênis Puma Feminino - Original,0.50
58231951,Tenis All Star Juvenil Preto Cano Alto Botinha,Tênis Infantil Unissex Star Preto Cano Alto,0.67
33919692,Tênis Feminino Olympikus Flash 910,Tênis Olympikus Easy Feminino Academia Caminhada Conforto650,0.55
